In [ ]:
import os

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter
from dmelon import plotting, utils

import xarray as xr

In [ ]:
settings = None

In [ ]:
settings = utils.load_json(settings)

In [ ]:
MONTH = settings["MONTH"]
FYEAR = settings["FYEAR"]
INIT_MONTH = settings["INIT_MONTH"]
DATA_DIR = settings["DATA_DIR"]
MONTH_DIR = os.path.join(DATA_DIR, f"{INIT_MONTH}.{MONTH}")
NC_DIR = os.path.join(DATA_DIR, str(FYEAR), f"{INIT_MONTH}.{MONTH}", "Data")
PLOTS_DIR = os.path.join(MONTH_DIR, "plots")

utils.check_folder(PLOTS_DIR)

In [ ]:
fcst_data = xr.open_dataset(os.path.join(NC_DIR, "fcst_data.nc")).fcst_data
fcst_data = fcst_data.where(fcst_data >= 0, 0)
fcst_data

In [ ]:
pisco = (
    xr.open_dataset(settings["PISCO_DATA"], decode_times=False)
    .rename({"X": "lon", "Y": "lat", "T": "time"})
    .load()
)
pisco.time.attrs["calendar"] = "360_day"
pisco = xr.decode_cf(pisco).Prec
pisco["time"] = pd.date_range("1981-01", "2016-12", freq="MS") + pd.DateOffset(days=14)
pisco = pisco.sel(time=slice("1981-10-01", "2016-05-01"))

In [ ]:
pisco_clim = pisco.groupby("time.month").mean("time")
pisco_clim

In [ ]:
no_neg = fcst_data.groupby("time.month") - pisco_clim

In [ ]:
HQ_BORDER = cfeature.NaturalEarthFeature(
    category="cultural",
    name="admin_0_countries",
    scale="50m",
    facecolor="white",
    edgecolor="grey",
)

In [ ]:
cdict = {
    "red": ((0.0, 1.0, 1.0), (0.4, 1.0, 1.0), (0.65, 0.0, 0.0), (1.0, 0.0, 0.0)),
    "green": ((0.0, 0.0, 0.0), (0.25, 1.0, 1.0), (0.7, 1.0, 1.0), (1.0, 0.0, 0.0)),
    "blue": ((0.0, 0.0, 0.0), (0.7, 0.0, 0.0), (1.0, 1.0, 1.0)),
    "alpha": (
        (0.0, 1.0, 1.0),
        (0.5, 0.0, 0.0),
        (1.0, 1.0, 1.0),
    ),
}
GnRd = colors.LinearSegmentedColormap("GnRd", cdict)

In [ ]:
year = FYEAR
cmap = GnRd

boundaries = np.arange(-160, 161, 20)
boundaries = boundaries[boundaries != 0]

norm = colors.BoundaryNorm(boundaries, cmap.N, extend="both")

fcst_sel = no_neg.sel(time=slice(f"{year-1}-{INIT_MONTH}-01", None))
fig, axs = plt.subplots(
    figsize=(6, 4),
    dpi=300,
    ncols=4,
    nrows=2,
    sharey=True,
    subplot_kw={"projection": ccrs.PlateCarree()},
)

for num, date in enumerate(fcst_sel.time.data[:8]):
    # for num, ax in enumerate(axs.ravel()):
    ax = axs.ravel().tolist()[num]
    p = ax.pcolormesh(
        fcst_sel.lon.data,
        fcst_sel.lat.data,
        fcst_sel.sel(time=date).data,  # * 100 / pisco_sub.isel(time=num),
        cmap=cmap,
        norm=norm,
        transform=ccrs.PlateCarree(),
    )
    ax.set_title(f"{pd.to_datetime(date):%Y-%m}", size=6)
    plotting.format_latlon(ax, ccrs.PlateCarree(), lon_step=5, lat_step=5)
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.COASTLINE)
    ax.set_extent((-81.25, -68.05, -18.75, 0.95), crs=ccrs.PlateCarree())
    ax.tick_params(axis="both", labelsize=5)
    ax.gridlines(linewidth=0.5, linestyle="--", alpha=0.5)
fig.colorbar(p, ax=axs.ravel().tolist(), extend="both", ticks=boundaries)
fig.suptitle("Precipitation Model Forecast Anomaly [mm]")
fig.text(0.4, 0.92, f"Model Initialized in {MONTH}", fontdict=dict(fontsize=5))
# fig.delaxes(ax=axs.ravel().tolist()[-1])
fig.savefig(
    os.path.join(PLOTS_DIR, "fcst_plot_mm.png"),
    bbox_inches="tight",
)

In [ ]:
year = FYEAR
cmap = GnRd

boundaries = np.arange(-100, 101, 10)
norm = colors.BoundaryNorm(boundaries, cmap.N, extend="both")

fcst_sel = (((fcst_data * 100).groupby("time.month") / pisco_clim) - 100).sel(
    time=slice(f"{year-1}-{INIT_MONTH}-01", None)
)
fig, axs = plt.subplots(
    figsize=(6, 4),
    dpi=300,
    ncols=4,
    nrows=2,
    sharey=True,
    subplot_kw={"projection": ccrs.PlateCarree()},
)

for num, date in enumerate(fcst_sel.time.data[:8]):
    # for num, ax in enumerate(axs.ravel()):
    ax = axs.ravel().tolist()[num]
    p = ax.pcolormesh(
        fcst_sel.lon.data,
        fcst_sel.lat.data,
        fcst_sel.sel(time=date).data,  # * 100 / pisco_sub.isel(time=num),
        cmap=cmap,
        norm=norm,
        transform=ccrs.PlateCarree(),
    )
    ax.set_title(f"{pd.to_datetime(date):%Y-%m}", size=6)
    plotting.format_latlon(ax, ccrs.PlateCarree(), lon_step=5, lat_step=5)
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.COASTLINE)
    ax.set_extent((-81.25, -68.05, -18.75, 0.95), crs=ccrs.PlateCarree())
    ax.tick_params(axis="both", labelsize=5)
    ax.gridlines(linewidth=0.5, linestyle="--", alpha=0.5)
fig.colorbar(p, ax=axs.ravel().tolist(), extend="both", ticks=boundaries[::2])
fig.suptitle("Precipitation Model Forecast Anomaly [%]")
fig.text(0.4, 0.92, f"Model Initialized in {MONTH}", fontdict=dict(fontsize=5))
# fig.delaxes(ax=axs.ravel().tolist()[-1])
fig.savefig(
    os.path.join(PLOTS_DIR, "fcst_plot_perc.png"),
    bbox_inches="tight",
)